In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
def convert_road_kartado(a):
    padrao = re.compile(r'\d', re.IGNORECASE)
    resultado = padrao.search(a)
    primeira_ocorrencia = resultado.start()
    primeira_string = a[:primeira_ocorrencia]
    restante = a[primeira_ocorrencia:]
    return primeira_string+'-'+restante

# Leitura do excel com os dados de municipios

In [5]:
data = pd.read_excel("Limites dos Municipios-Cadastro Kartado (1).xlsx",sheet_name=0)

In [6]:
# data = data.drop(data.index[0])

In [7]:
data

,CONCESSIONÁRIA,BR,ESTADO,MUNICIPIO,KM INIC,KM FINAL,DISTÂNCIA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LITORAL SUL,116,PR,QUATRO BARRAS,71.000,77.310,6.310
2,LITORAL SUL,116,PR,PIRAQUARA,77.310,91.520,14.210
3,LITORAL SUL,116,PR,SÃO JOSÉ DOS PINHAIS,91.520,108.160,16.640
4,LITORAL SUL,116,PR,CURITIBA,108.160,116.000,7.840
...,...,...,...,...,...,...,...
232,INTERVIAS,SP 330,SP,Santa Cruz da Conceição,196.400,201.662,5.262
233,INTERVIAS,SP 330,SP,Pirassununga,201.662,219.890,18.228
234,INTERVIAS,SP 330,SP,Porto Ferreira,219.890,232.960,13.070
235,INTERVIAS,SP 330,SP,Santa Rita do Passa Quatro,232.960,240.500,7.540


# Arrumar as rodovias pra ficarem de acordo com a unidade

In [6]:
values=[]
for index, row in data.iterrows():
    if row['CONCESSIONÁRIA'] == 'PLANALTO SUL':
        if 'SP' in row['BR'] and (not 'SPA' in row['BR'] and not 'SPI' in row['BR']):
            row['BR'] = 'SP'+'-'+row['BR'].split('SP')[1]
        elif 'SPA' in row['BR']:
            row['BR'] = 'SPA'+'-'+row['BR'].split('SPA')[1]
        elif 'SPI' in row['BR']:
            row['BR'] = 'SPI'+'-'+row['BR'].split('SPI')[1]
        values.append({"CONCESSIONÁRIA":'VIAPAULISTA',
                       'BR':row['BR'],
                       'MUNICIPIO':row['MUNICIPIO'],
                      'KM INIC':row['KM INIC'],
                      'KM FINAL':row['KM FINAL']})
#         print(row['BR'])

TypeError: argument of type 'int' is not iterable

In [7]:
values

[{'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-249',
  'MUNICIPIO': 'Coronel Macedo',
  'KM INIC': 144.115,
  'KM FINAL': 149.0},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-249',
  'MUNICIPIO': 'Taquarituba',
  'KM INIC': 149.0,
  'KM FINAL': 159.0},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Araraquara',
  'KM INIC': 83.2,
  'KM FINAL': 95.16},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Boa Esperança do Sul',
  'KM INIC': 95.16,
  'KM FINAL': 122.22},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Dourado',
  'KM INIC': 122.22,
  'KM FINAL': 124.02},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Bocaina',
  'KM INIC': 124.02,
  'KM FINAL': 138.61},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Jaú',
  'KM INIC': 138.61,
  'KM FINAL': 147.3},
 {'CONCESSIONÁRIA': 'VIAPAULISTA',
  'BR': 'SP-255',
  'MUNICIPIO': 'Jaú',
  'KM INIC': 155.77,
  'KM FINAL': 167.02},
 {'C

In [17]:
convert_road_kartado('SPA004/257')

'SPA-004/257'

# Gera a lógica para o autofill de municipios

In [18]:
import json
exemplo = {}
chaves_finais = []

# Loop através das linhas do seu DataFrameBR-101
for index, row in data.iterrows():
# for row in values:
    if row['CONCESSIONÁRIA'] == 'VIAPAULISTA':
        road_name=convert_road_kartado(str(row["BR"]).strip())
        if row["KM INIC"] == 0:
            exemplo = {
                        "and": [{
                            "==": [
                                {"var": "roadName"},
                                road_name
                            ]
                        },
                            {
                                "<=": [
                                    {"var": "km"},
                                    row["KM FINAL"]
                                ]
                            }
                        ]
                    }
        else:
            exemplo = {
                        "and": [{
                            "==": [
                                {"var": "roadName"},
                                road_name
                            ]
                        },{
                                ">=": [
                                    {"var": "km"},
                                    row["KM INIC"]
                                ]
                            },
                            {
                                "<=": [
                                    {"var": "km"},
                                    row["KM FINAL"]
                                ]
                            }
                        ]
                    }


        chaves_finais.append(exemplo)
        chaves_finais.append(row["MUNICIPIO"].strip())
chaves_finais.append("NA")
# Monta o JSON final com todas as condições
json_final = {
    "if": chaves_finais
}

# Converte o JSON para uma string formatada
json_string = json.dumps(json_final)

print(json_string)


{"if": [{"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 144.115]}, {"<=": [{"var": "km"}, 149.0]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 149.0]}, {"<=": [{"var": "km"}, 159.0]}]}, "Taquarituba", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 83.2]}, {"<=": [{"var": "km"}, 95.16]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 95.16]}, {"<=": [{"var": "km"}, 122.22]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 122.22]}, {"<=": [{"var": "km"}, 124.02]}]}, "Dourado", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 124.02]}, {"<=": [{"var": "km"}, 138.61]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 138.61]}, {"<=": [{"var": "km"}, 147.3]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var

# Teste da lógica criada

In [9]:
from json_logic import jsonLogic

jsonLogic(json_final,{'km':681,'roadName':'BR-381/MG'})

'PERDÕES'

In [15]:
data

,Rodovia,km inicial,km final,EAF
0,ASS 245,0.0,7.139,EAF5
1,AVM 030,0.0,16.230,EAF10
2,AVM 075,0.0,3.171,EAF10
3,CNP 050,0.0,14.156,EAF5
4,IBM 122,0.0,9.868,EAF5
...,...,...,...,...
58,SPA 652/270,0.0,0.710,EAF10
59,SPT 035,0.0,4.376,EAF5
60,TDS 152,0.0,2.136,EAF10
61,TDS 328,0.0,1.717,EAF10


# Lógica com between

In [18]:
import json
exemplo = {}
chaves_finais = []

# Loop através das linhas do seu DataFrame
for index, row in data.iterrows():
    if row["km inicial"] == 0:
        exemplo = {
                    "and": [{
                        "==": [
                            {"var": "roadName"},
                            row["Rodovia"]
                        ]
                    },
                        {
                            "<=": [
                                {"var": "km"},
                                row["km final"]
                            ]
                        }
                    ]
                }
    else:
        exemplo = {
                    "and": [{
                        "==": [
                            {"var": "roadName"},
                            row["Rodovia"]
                        ]
                    },{
                            "<=": [row["km inicial"],
                                {"var": "km"},
                                row["km final"]
                            ]
                        }
                    ]
                }


    chaves_finais.append(exemplo)
    chaves_finais.append(row["EAF"])
chaves_finais.append("NA")
# Monta o JSON final com todas as condições
json_final = {
    "if": chaves_finais
}

# Converte o JSON para uma string formatada
json_string = json.dumps(json_final)

print(json_string)

{"if": [{"and": [{"==": [{"var": "roadName"}, "ASS 245"]}, {"<=": [{"var": "km"}, 7.139]}]}, "EAF5", {"and": [{"==": [{"var": "roadName"}, "AVM 030"]}, {"<=": [{"var": "km"}, 16.23]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "AVM 075"]}, {"<=": [{"var": "km"}, 3.171]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "CNP 050"]}, {"<=": [{"var": "km"}, 14.156]}]}, "EAF5", {"and": [{"==": [{"var": "roadName"}, "IBM 122"]}, {"<=": [{"var": "km"}, 9.868]}]}, "EAF5", {"and": [{"==": [{"var": "roadName"}, "IBM 324"]}, {"<=": [{"var": "km"}, 0.687]}]}, "EAF5", {"and": [{"==": [{"var": "roadName"}, "MBP 032"]}, {"<=": [{"var": "km"}, 15.335]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "MBP 173"]}, {"<=": [{"var": "km"}, 15.922]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "MBP 322"]}, {"<=": [{"var": "km"}, 4.707]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "MPR 311"]}, {"<=": [{"var": "km"}, 14.62]}]}, "EAF10", {"and": [{"==": [{"var": "roadName"}, "MRC 060"]}, {"

# opcionais/testes

# Versão que não fica vazia caso coloque um km errado

In [159]:
import json

chaves_finais = []

# Loop através das linhas do seu DataFrame
for index, row in data.iterrows():
    exemplo = {
        "and": [
            {
                "if": [{
                    "==": [
                        {"var": "roadName"},
                        row["Rodovia"]
                    ]
                },{
                "and": [
                    {
                        ">=": [
                            {"var": "km"},
                            row["km inicial"]
                        ]
                    },
                    {
                        "<=": [
                            {"var": "km"},
                            row["km final"]
                        ]
                    }
                ]
            }]
            },
            row["EAF"]
        ]
    }
    
    chaves_finais.append(exemplo)

# Monta o JSON final com todas as condições
json_final = {
    "if": [
					{
						"missing": [
							"km"
						]
					},
					"",{
    "or": chaves_finais}]
}

# Converte o JSON para uma string formatada
json_string = json.dumps(json_final)

print(json_string)


{"if": [{"missing": ["km"]}, "", {"or": [{"and": [{"if": [{"==": [{"var": "roadName"}, "SP-322"]}, {"and": [{">=": [{"var": "km"}, 307.59]}, {"<=": [{"var": "km"}, 390.5]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-330"]}, {"and": [{">=": [{"var": "km"}, 318.5]}, {"<=": [{"var": "km"}, 450.11]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-328"]}, {"and": [{">=": [{"var": "km"}, 323.13]}, {"<=": [{"var": "km"}, 337.01]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-328"]}, {"and": [{">=": [{"var": "km"}, 468.39]}, {"<=": [{"var": "km"}, 475.74]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-327/330"]}, {"and": [{">=": [{"var": "km"}, 0.0]}, {"<=": [{"var": "km"}, 5.5]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-355/330"]}, {"and": [{">=": [{"var": "km"}, 0.0]}, {"<=": [{"var": "km"}, 6.077]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-410/330"]}, {"and": [{">=": 

# 

In [159]:
import json

chaves_finais = []

# Loop através das linhas do seu DataFrame
for index, row in data.iterrows():
    exemplo = {
        "and": [
            {
                "if": [{
                    "==": [
                        {"var": "roadName"},
                        row["Rodovia"]
                    ]
                },{
                "and": [
                    {
                        ">=": [
                            {"var": "km"},
                            row["km inicial"]
                        ]
                    },
                    {
                        "<=": [
                            {"var": "km"},
                            row["km final"]
                        ]
                    }
                ]
            }]
            },
            row["EAF"]
        ]
    }
    
    chaves_finais.append(exemplo)

# Monta o JSON final com todas as condições
json_final = {
    "if": [
					{
						"missing": [
							"km"
						]
					},
					"",{
    "or": chaves_finais}]
}

# Converte o JSON para uma string formatada
json_string = json.dumps(json_final)

print(json_string)


{"if": [{"missing": ["km"]}, "", {"or": [{"and": [{"if": [{"==": [{"var": "roadName"}, "SP-322"]}, {"and": [{">=": [{"var": "km"}, 307.59]}, {"<=": [{"var": "km"}, 390.5]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-330"]}, {"and": [{">=": [{"var": "km"}, 318.5]}, {"<=": [{"var": "km"}, 450.11]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-328"]}, {"and": [{">=": [{"var": "km"}, 323.13]}, {"<=": [{"var": "km"}, 337.01]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SP-328"]}, {"and": [{">=": [{"var": "km"}, 468.39]}, {"<=": [{"var": "km"}, 475.74]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-327/330"]}, {"and": [{">=": [{"var": "km"}, 0.0]}, {"<=": [{"var": "km"}, 5.5]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-355/330"]}, {"and": [{">=": [{"var": "km"}, 0.0]}, {"<=": [{"var": "km"}, 6.077]}]}]}, "EAF6"]}, {"and": [{"if": [{"==": [{"var": "roadName"}, "SPA-410/330"]}, {"and": [{">=": 

In [16]:
from json_logic import jsonLogic

jsonLogic({
  "min": {
    "map": [
      {
        "filter": [
          {"var": "element"},
          {
            "and": [
              {"in": [{"var": "inspected_element"}, "1"]},
              {"!=": [{"var": "mouth_obs_mon_inner"}, False]}
            ]
          }
        ]
      },
      {"+": [{"var": "mouth_obs_mon_inner"}]}
    ]
  }
},{
  "entry": "1",
  "element": [
    {
      "order_of_element": "1",
      "inspected_element": "1",
      "ale_damage_mon_inner": "1",
      "sol_damage_mon_inner": "2",
      "drown_mouth_mon_inner": "1",
      "forehead_damage_two_mon_inner": "1"
    },
    {
      "inspected_element": "1",
      "mouth_obs_mon_inner": "1",
      "ale_damage_mon_inner": "1",
      "sol_damage_mon_inner": "2",
      "drown_mouth_mon_inner": "1",
      "forehead_damage_two_mon_inner": "3"
    },
    {
      "inspected_element": "1",
      "ale_damage_mon_inner": "1",
      "sol_damage_mon_inner": "2",
      "drown_mouth_mon_inner": "1",
      "forehead_damage_two_mon_inner": "3"
    },
    {
      "drown_body_inner": "2",
      "damage_body_inner": "3",
      "inspected_element": "2",
      "body_assoread_inner": "1",
      "desalignment_body_inner": "1",
      "joint_infiltrated_inner": "1"
    },
    {
      "drown_body_inner": "1",
      "damage_body_inner": "2",
      "inspected_element": "2",
      "desalignment_body_inner": "1",
      "joint_infiltrated_inner": "1"
    }
  ],
  "cod_monit": "nulkl",
  "drown_body": [],
  "damage_body": [],
  "locate_hole": False,
  "stretch_hole": False,
  "body_assoread": [],
  "dissipe_speed": False,
  "mouth_obs_mon": [],
  "nothing_to_do": False,
  "recover_slope": False,
  "recover_walls": False,
  "ale_damage_mon": [],
  "recover_bodies": False,
  "recover_mouths": False,
  "sol_damage_mon": [],
  "drown_mouth_mon": [],
  "recover_gadgets": False,
  "request_studies": False,
  "execute_cleaning": False,
  "desalignment_body": [],
  "joint_infiltrated": [],
  "channel_water_course": False,
  "investigate_influence": False,
  "recover_or_substitute": False,
  "forehead_damage_two_mon": []
})

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'